In [169]:
from vnstock3 import Vnstock
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import datetime as dt
import statistics 
from datetime import datetime, timedelta
import copy
# from add_indicators import calculate_indicators
# from add_signals import calculate_all_signals

In [170]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
import pickle

In [171]:
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

In [172]:
stock = Vnstock().stock(source='VCI')
vn30_tickers = stock.listing.symbols_by_group('VN30').values
vn30_tickers = np.delete(vn30_tickers, np.where(vn30_tickers == 'SSB')) # SSB chỉ có giá trị từ 2021 (quá ít data để sử dụng)
vn30_tickers

2025-01-04 10:43:27 - vnstock3.common.vnstock - INFO - Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
2025-01-04 10:43:27 - vnstock3.common.data.data_explorer - INFO - Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
2025-01-04 10:43:27 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


array(['ACB', 'BCM', 'BID', 'BVH', 'CTG', 'FPT', 'GAS', 'GVR', 'HDB',
       'HPG', 'MBB', 'MSN', 'MWG', 'PLX', 'POW', 'SAB', 'SHB', 'SSI',
       'STB', 'TCB', 'TPB', 'VCB', 'VHM', 'VIB', 'VIC', 'VJC', 'VNM',
       'VPB', 'VRE'], dtype=object)

In [173]:
global_start = '2018-12-01'
global_end = '2025-01-01'

market_data = Vnstock().stock(symbol = 'VNINDEX', source = 'VCI').\
quote.history(start = global_start, end = global_end, interval = '1M')
market_data['profit'] = (market_data['close'] - market_data['close'].shift(1)) / market_data['close'].shift(1) * 100
market_data.dropna(inplace = True)
bond_data = pd.read_csv('Vietnam_10_year_Government_Bond.csv')
bond_data.columns = ['time', 'close', 'open',' high', 'low', 'change']
bond_data['time'] = pd.to_datetime(bond_data['time'])
bond_data['profit'] = ((1 + bond_data['close']/100) ** (1/12) - 1) * 100

2025-01-04 10:43:27 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [176]:
quarter_start_months = {1: '01', 2: '04', 3: '07', 4: '10'}

class Company:
    def __init__(self, ticker, start, end):
        self.ticker = ticker
        stock = Vnstock().stock(symbol = self.ticker, source = 'VCI')
        self.history = stock.quote.history(start = start, end = end, interval = '1M')
        
        self.history.loc[:, ['open', 'close', 'high', 'low']] = self.history[['open', 'close', 'high', 'low']] * 1000
        self.ratio = stock.finance.ratio(period='quarter', lang='en', dropna=True)
        self.balance = stock.finance.balance_sheet(period='quarter', lang='en', dropna=True)
        self.history['shares'] = np.nan

        # Tính market cap, và book value
        meta = self.ratio['Meta']
        meta['date'] = np.nan
        stats = self.ratio['Chỉ tiêu định giá']
        for i in meta.index:
            meta.loc[i, 'date'] = f'{meta['yearReport'][i]}-{quarter_start_months[meta['lengthReport'][i]]}-01'
        meta['date'] = pd.to_datetime(meta['date'])
        self.balance['date'] = np.nan
        for i in self.balance.index:
            self.balance.loc[i, 'date'] = f'{self.balance['yearReport'][i]}-{quarter_start_months[self.balance['lengthReport'][i]]}-01'
        self.balance.loc[:, 'date'] = pd.to_datetime(self.balance['date'])

        for date in meta['date'].index:
            if (meta.loc[date, 'date'] < pd.to_datetime(end)) and (meta.loc[date, 'date'] > pd.to_datetime(start)):
                temp_day = 0
                while True:
                    if len(self.history.loc[self.history['time'] == meta.loc[date, 'date'] + timedelta(days = temp_day), 'shares']) == 1:
                        self.history.loc[self.history['time'] == meta.loc[date, 'date'] + timedelta(days = temp_day), 'shares'] = stats.loc[date, 'Outstanding Share (Mil. Shares)']
                        self.history.loc[self.history['time'] == meta.loc[date, 'date'] + timedelta(days = temp_day), 'book_value'] = self.balance.loc[date, "OWNER'S EQUITY(Bn.VND)"]
                        break
                    elif len(self.history.loc[self.history['time'] == meta.loc[date, 'date'] - timedelta(days = temp_day), 'shares']) == 1:
                        self.history.loc[self.history['time'] == meta.loc[date, 'date'] - timedelta(days = temp_day), 'shares'] = stats.loc[date, 'Outstanding Share (Mil. Shares)']
                        self.history.loc[self.history['time'] == meta.loc[date, 'date'] - timedelta(days = temp_day), 'book_value'] = self.balance.loc[date, "OWNER'S EQUITY(Bn.VND)"]
                        break
                    temp_day += 1
        self.history = self.history.fillna(method = 'ffill').fillna(method = 'bfill')
        self.history['market_cap'] = self.history['shares'] * self.history['close']

        # Tính Book to Market Ratio
        self.history.loc[:, 'btm'] = self.history['book_value'] / self.history['market_cap']

        self.history.loc[:, 'hml'] = self.history['btm'].apply(lambda x : 'high' if x >= 1 else 'low')

        self.history.loc[:, 'monthly_profit'] = (self.history['close'] - self.history['close'].shift(1)) / self.history['close'].shift(1) * 100

        self.history = self.history.iloc[1:]

In [177]:

class Portfolio:
    def __init__(self, tickers, start, end, data = None):
        self.tickers = tickers  
        if data is None:
            self.data = {}
        else:
            self.data = data
        self.betas = []
        self.start = start
        self.end = end
        
    def create_profiles(self):
        for ticker in self.tickers:
            print(ticker)
            self.data[ticker] = Company(ticker, start = self.start, end = self.end)
        
    def create_sub_portfolio(self, sub_tickers):
        new_portfolio = Portfolio(tickers = sub_tickers, start = self.start, end = self.end)
        for ticker in sub_tickers:
            new_portfolio.data[ticker] = copy.deepcopy(self.data[ticker])
        return new_portfolio
    
    def evaluate_smb(self):
        aggregate_market_cap = pd.DataFrame([], columns= self.tickers)

        for ticker in self.tickers:
            self.data[ticker].history['smb'] = np.nan

        for index in self.data[self.tickers[0]].history.index:
            for ticker in self.tickers: 
                aggregate_market_cap.loc[index, ticker] = self.data[ticker].history.loc[index, 'market_cap']
                
        for index in aggregate_market_cap.index:
            percentiles = np.percentile(aggregate_market_cap.loc[index].values, [10, 30]) 
            for ticker in self.tickers:
                x = aggregate_market_cap.loc[index, ticker] 
                if x <= percentiles[0]:
                    self.data[ticker].history.loc[index, 'smb'] = 'small' 
                elif x <= percentiles[1]:
                    self.data[ticker].history.loc[index, 'smb'] = 'mid'
                else:
                    self.data[ticker].history.loc[index, 'smb'] = 'big'

    def setup_regression(self, market_data, bond_data):
        regression_data = pd.DataFrame([], columns= ['time', 'excess_return', 'mp', 'smb', 'hml'])

        for index in self.data[self.tickers[0]].history.index:
            date = self.data[self.tickers[0]].history.loc[index, 'time']
            smb = 0
            hml = 0

            temp_day = 0
            print(date)
            while True:
                try:         # Because sometimes the date is off by 1 or 2 days
                    rf = bond_data[bond_data['time'] == date + timedelta(days = temp_day)]['profit'].values[0]
                    break
                except:
                    try:
                        rf = bond_data[bond_data['time'] == date - timedelta(days = temp_day)]['profit'].values[0]
                        break
                    except:
                        temp_day += 1
                    
            mp = market_data.loc[index, 'profit'] - rf

            for ticker in self.tickers:
                if self.data[ticker].history.loc[index, 'smb'] == 'small':
                    smb += self.data[ticker].history.loc[index, 'monthly_profit']
                elif self.data[ticker].history.loc[index, 'smb'] == 'big':
                    smb -= self.data[ticker].history.loc[index, 'monthly_profit']
                if self.data[ticker].history.loc[index, 'hml'] == 'high':
                    hml += self.data[ticker].history.loc[index, 'monthly_profit']
                else:
                    hml -= self.data[ticker].history.loc[index, 'monthly_profit']
            
            regression_data.loc[index, 'smb'] = smb
            regression_data.loc[index, 'hml'] = hml
            regression_data.loc[index, 'mp'] = mp
            regression_data.loc[index, 'rf'] = rf
            regression_data.loc[index, 'time'] = date
        # regression_data['excess_return'] = regression_data['excess_return'].shift(-1)

        # regression_data.dropna(inplace = True)
        return regression_data
        
    def ff3_metrics(self, og_regression_data, use_data = 'newest', cutoff = None):
        # TÍNH EXCESS RETURN CHO TỪNG PORTFOLIO LẺ
        regression_data = copy.deepcopy(og_regression_data)

        for index in regression_data.index:
            total_return = np.mean([self.data[ticker].history.loc[index, 'monthly_profit'] for ticker in self.tickers]) - regression_data.loc[index, 'rf']
            regression_data.loc[index, 'excess_return'] = total_return

        if cutoff is None:
            cutoff = round((len(regression_data) * 0.8))
            
        train_set = regression_data[:cutoff]
        test_set = regression_data[cutoff:]

        y_train = train_set['excess_return']
        
        x_train = train_set.drop(columns = ['time', 'excess_return', 'rf'])

        y_test = test_set['excess_return']
        if use_data == 'newest':
            x_test = test_set.drop(columns = ['time', 'excess_return', 'rf'])

        elif use_data == 'average':
            x_test = test_set.drop(columns = ['time', 'excess_return', 'rf'])
            for column in x_test.columns:
                m = np.mean(x_train[column])
                for index in x_test.index:
                    x_test.loc[index, column] = m

        model = LinearRegression()
        # model = MLPRegressor(hidden_layer_sizes=(10,), max_iter=1000, random_state=42)

        model.fit(x_train, y_train)
        
        y_train_pred = model.predict(x_train)
        
        mse_train = mean_squared_error(y_train, y_train_pred)

        y_pred = model.predict(x_test)
        mse_test = mean_squared_error(y_test, y_pred)
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_pred)

        return mse_train, mse_test, r2_train, r2_test, model.coef_

In [ ]:
monthly_data = {}
for ticker in vn30_tickers:
    monthly_data[ticker] = Company(ticker, start = global_start, end = global_end)

2025-01-04 10:43:28 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2025-01-04 10:43:30 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2025-01-04 10:43:32 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2025-01-04 10:43:35 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2025-01-04 10:43:37 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2025-01-04 10:43:39 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2025-01-04 10:43:40 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2025-01-04 10:43:42 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất 

In [180]:
full_portfolio = Portfolio(vn30_tickers, start = global_start, end = global_end, data = monthly_data)

# full_portfolio.create_profiles()

In [181]:
aggregate_market_cap = full_portfolio.evaluate_smb()

In [182]:
regression_data = full_portfolio.setup_regression(market_data, bond_data)

2019-01-31 00:00:00
2019-02-28 00:00:00
2019-03-31 00:00:00
2019-04-30 00:00:00
2019-05-31 00:00:00
2019-06-30 00:00:00
2019-07-31 00:00:00
2019-08-31 00:00:00
2019-09-30 00:00:00
2019-10-31 00:00:00
2019-11-30 00:00:00
2019-12-31 00:00:00
2020-01-31 00:00:00
2020-02-29 00:00:00
2020-03-31 00:00:00
2020-04-30 00:00:00
2020-05-31 00:00:00
2020-06-30 00:00:00
2020-07-31 00:00:00
2020-08-31 00:00:00
2020-09-30 00:00:00
2020-10-31 00:00:00
2020-11-30 00:00:00
2020-12-31 00:00:00
2021-01-31 00:00:00
2021-02-28 00:00:00
2021-03-31 00:00:00
2021-04-30 00:00:00
2021-05-31 00:00:00
2021-06-30 00:00:00
2021-07-31 00:00:00
2021-08-31 00:00:00
2021-09-30 00:00:00
2021-10-31 00:00:00
2021-11-30 00:00:00
2021-12-31 00:00:00
2022-01-31 00:00:00
2022-02-28 00:00:00
2022-03-31 00:00:00
2022-04-30 00:00:00
2022-05-31 00:00:00
2022-06-30 00:00:00
2022-07-31 00:00:00
2022-08-31 00:00:00
2022-09-30 00:00:00
2022-10-31 00:00:00
2022-11-30 00:00:00
2022-12-31 00:00:00
2023-01-31 00:00:00
2023-02-28 00:00:00


In [185]:
cutoff = 36

In [186]:
price_df = pd.DataFrame([], columns= vn30_tickers, index = full_portfolio.data[vn30_tickers[0]].history.index)
for index in price_df.index:
    for ticker in vn30_tickers:
        price_df.loc[index, ticker] = full_portfolio.data[ticker].history.loc[index, 'monthly_profit']
price_df['average_monthly_profit'] = price_df.mean(axis = 1)
price_df['date'] = full_portfolio.data[vn30_tickers[0]].history['time']
vn30_market_growth = 1
for index in price_df[cutoff:].index:
    vn30_market_growth = vn30_market_growth * (1 + price_df.loc[index, 'average_monthly_profit'] / 100)
vn30_market_growth = (vn30_market_growth - 1) * 100
vn30_market_growth

-0.7035880610744449

In [ ]:
results = []

for ticker in vn30_tickers:
    temp = full_portfolio.create_sub_portfolio(sub_tickers= [ticker])
    
    _,_,_,_, [mp, smb, hml] = temp.ff3_metrics(regression_data, cutoff = cutoff)
    results.append([ticker, mp, smb, hml])

    del temp

In [190]:
mp_results = sorted(results, key = lambda x : x[1])
smb_results = sorted(results, key = lambda x : x[2])
hml_results = sorted(results, key = lambda x : x[3])

In [212]:
import itertools

perms = [list(i) for i in list(itertools.product([0, 1], repeat=3))]

portfolios = []
n = 2
duplicate = False
for n in range(2,5):
    for perm in perms:
        portfolio = []
        if perm[0] == 0:
            portfolio.extend([i[0] for i in mp_results][:n])
        else:
            portfolio.extend([i[0] for i in mp_results[::-1]][:n])
        if perm[1] == 0:
            portfolio.extend([i[0] for i in smb_results][:n])
        else:
            portfolio.extend([i[0] for i in smb_results[::-1]][:n])

        if perm[2] == 0:
            portfolio.extend([i[0] for i in hml_results][:n])
        else:
            portfolio.extend([i[0] for i in hml_results[::-1]][:n])
        if duplicate:
            portfolios.append(portfolio)
        else:
            portfolios.append(list(set(portfolio)))

In [213]:
monthly_test_data = pd.DataFrame([], columns = vn30_tickers)
for index in full_portfolio.data[vn30_tickers[0]].history.index:
    for ticker in vn30_tickers:
        monthly_test_data.loc[index, ticker] = full_portfolio.data[ticker].history.loc[index, 'monthly_profit']
monthly_test_data['date'] = full_portfolio.data['FPT'].history['time']
monthly_test_data = monthly_test_data.iloc[cutoff-1:].reset_index(drop = True)

In [214]:
def total_profit(history, col = 'profit'):
    total_profit = 1
    for index in history.index:
        total_profit = (total_profit + (total_profit * history.loc[index, col]) /100)
    return (total_profit - 1) * 100

In [215]:
def backtest_basic(data, portfolio):  #Create a dataframe with profit of the input portfolio
    temp_df = pd.DataFrame([], columns = portfolio)
    for index in data.index:
        for ticker in portfolio:
            temp_df.loc[index, ticker] = data.loc[index, ticker]
    temp_df['portfolio_profit'] = temp_df.mean(axis = 1)
    temp_df['market_growth'] = data.drop(columns = ['date']).mean(axis = 1)
    temp_df['date'] = full_portfolio.data[vn30_tickers[0]].history['time'].iloc[cutoff:]
    return temp_df

backtest_results = {}
for portfolio in portfolios:
    backtest_results[tuple(portfolio)] = backtest_basic(monthly_test_data, portfolio)
# backtest_results: dictionary of profits for each portfolio

metrics_df = pd.DataFrame(list(zip(perms, portfolios)), columns = ['strategy', 'portfolio'])

metrics_df['ap'] = 0
metrics_df['al'] = 0
metrics_df['np'] = 0
metrics_df['wr'] = 0

metrics_df['market_profit'] = 0

# Average profit, average loss, net profit, win rate

for portfolio in portfolios:
    backtest_result = backtest_results[tuple(portfolio)]

    metrics_df.loc[metrics_df['portfolio'].apply(lambda x : x == portfolio), 'ap'] = backtest_result.loc[backtest_result['portfolio_profit'] > 0, 'portfolio_profit'].mean()
    metrics_df.loc[metrics_df['portfolio'].apply(lambda x : x == portfolio), 'al'] = backtest_result.loc[backtest_result['portfolio_profit'] < 0, 'portfolio_profit'].mean()
    metrics_df.loc[metrics_df['portfolio'].apply(lambda x : x == portfolio), 'np'] = backtest_result['portfolio_profit'].mean()
    metrics_df.loc[metrics_df['portfolio'].apply(lambda x : x == portfolio), 'wr'] = len(backtest_result.loc[backtest_result['portfolio_profit'] > 0, 'portfolio_profit']) / len(backtest_result) * 100
    metrics_df.loc[metrics_df['portfolio'].apply(lambda x : x == portfolio), 'tp'] = total_profit(backtest_result, col = 'portfolio_profit')
metrics_df['market_profit'] = total_profit(backtest_basic(monthly_test_data, vn30_tickers), col = 'portfolio_profit')

In [216]:
metrics_df

,strategy,portfolio,ap,al,np,wr,market_profit,tp
0,"[0, 0, 0]","[MSN, GAS, FPT, SHB, TPB]",4.141166,-4.842458,0.256355,56.756757,1.587534,3.290055
1,"[0, 0, 1]","[MSN, MWG, FPT, POW, SHB]",5.290713,-5.031534,0.269079,51.351351,1.587534,1.315146
2,"[0, 1, 0]","[MSN, SSI, HDB, GAS, SHB, TPB]",4.940579,-6.558080,-0.031814,56.756757,1.587534,-9.567561
3,"[0, 1, 1]","[MSN, SSI, MWG, HDB, POW, SHB]",5.246322,-6.934847,-0.021211,56.756757,1.587534,-10.585932
4,"[1, 0, 0]","[MSN, SSI, HDB, GAS, FPT, TPB]",4.769242,-6.239821,0.603651,62.162162,1.587534,15.652303
5,"[1, 0, 1]","[MSN, SSI, MWG, HDB, FPT, POW]",5.648450,-5.993128,0.614254,56.756757,1.587534,14.148770
6,"[1, 1, 0]","[SSI, HDB, GAS, TPB]",5.185282,-6.511480,0.443351,59.459459,1.587534,7.490473
7,"[1, 1, 1]","[SSI, MWG, HDB, POW]",5.863293,-6.633542,0.459256,56.756757,1.587534,6.136082


In [217]:
# Explanation of coefficients:

# MP: Higher beta is good in bullish markets, lower beta is good in bearish market.
# SMB: Higher is more risky. Lower is safer
# HML: Higher is good in economic recoveries or in mature markets. Lower is good in strong bull market.

# 1 is high, 0 is low

In [218]:
# Same as backtest_basic(), but more detailed
def backtest_single(monthly_data, portfolio, log = False):
    budget_single = 10000000
    temp_df = pd.DataFrame([], columns = portfolio)

    for index in monthly_data.index:
        for ticker in portfolio:
            temp_df.loc[index, ticker] = monthly_data.loc[index, ticker]
    temp_df['portfolio_profit'] = temp_df.mean(axis = 1)
    temp_df['date'] = full_portfolio.data[vn30_tickers[0]].history['time'].iloc[cutoff:]

    for index in temp_df.index:
        budget_single = budget_single * (1 + temp_df.loc[index, 'portfolio_profit'] / 100)
        if log:
            print(f"Date: {temp_df.loc[index, 'date']}    Portfolio changed by: {temp_df.loc[index, 'portfolio_profit']}%    Current total assets: {budget_single}")
    profit_single  = ((budget_single - 10000000) / 10000000) * 100
    if log:
        print(f'Trading finished. Total profit: {profit_single}')
    return temp_df, profit_single

# 
def backtest_diverse(monthly_data, portfolios, log = False):
    temp_dfs = []
    budget = 10000000

    history = pd.DataFrame([], columns = ['date', 'profit', 'total_value'])

    for portfolio in portfolios:
        temp_df, _ = backtest_single(monthly_data, portfolio)
        temp_dfs.append(temp_df)
    for index in temp_df.index:
        profit_month = np.mean([i.loc[index, 'portfolio_profit'] for i in temp_dfs])
        budget = budget * (1 + profit_month / 100)
        if log:
            print(f"Date: {temp_df.loc[index, 'date']}    Portfolio changed by: {profit_month}%    Current total assets: {budget}")
        
        history.loc[index, 'profit'] = profit_month
        history.loc[index, 'total_value'] = budget

    profit_full  = ((budget - 10000000) / 10000000) * 100
    if log:
        print(f'Trading finished. Total profit: {profit_full}')
    
    metrics = pd.DataFrame([], columns = ['percentage'])
    metrics.loc['ap'] = history.loc[history['profit'] > 0, 'profit'].mean()
    metrics.loc['al'] = history.loc[history['profit'] < 0, 'profit'].mean()
    metrics.loc['ar'] = history['profit'].mean()
    metrics.loc['wr'] = len(history[history['profit'] > 0]) / len(history)
    history = history.reset_index(drop= True)
    history['date'] = temp_df['date'].reset_index(drop = True)
    
    return history, metrics

In [238]:
history, metrics = backtest_diverse(monthly_test_data, portfolios)

In [240]:
print('Downtrend: 2022 - 2023')
print(f'VN30 average: {total_profit(monthly_test_data[1:13])}')
print(f'Strategy profit: {total_profit(history[0:12])}\n')

print('Uptrend 1: 2023 - 2024:')
print(f'VN30 average: {total_profit(monthly_test_data[13:25])}')
print(f'Strategy profit: {total_profit(history[12:24])}\n')

print('Uptrend 2: 2024 - 2025:')
print(f'VN30 average: {total_profit(monthly_test_data[25:])}')
print(f'Strategy profit: {total_profit(history[24:])}\n')

Downtrend: 2022 - 2023
VN30 average: -26.072728055416906
Strategy profit: -26.966113252502154

Uptrend 1: 2023 - 2024:
VN30 average: 14.455056701177572
Strategy profit: 12.49657674687139

Uptrend 2: 2024 - 2025:
VN30 average: 17.352917371112152
Strategy profit: 28.136102204143132



In [220]:
metrics

,percentage
ap,4.764660
al,-6.123218
ar,0.350655
wr,0.594595


In [221]:
total_profit(history)

np.float64(5.277419161936714)

In [242]:
def backtest_adaptive(monthly_data, portfolios, log = False):
    budget = 10000000
    temp_dfs = []

    for portfolio in portfolios:
        temp_df, _ = backtest_single(monthly_data, portfolio)
        temp_dfs.append([portfolio, temp_df])
    
    history = pd.DataFrame([], columns = ['date', 'profit'])
    chosen_portfolio = sorted(temp_dfs, key = lambda x : x[1].iloc[0]['portfolio_profit'], reverse= True)[0][0] 
    
    for i in range(1, len(monthly_data)):
        profit = monthly_data.iloc[i][chosen_portfolio].mean()
        budget = budget * (1 + profit / 100)
        if log:
            print(f"{i}. Date: {monthly_data.iloc[i]['date']}    Portfolio: {chosen_portfolio}    Portfolio changed by: {profit}%    Current total assets: {budget}")
        chosen_portfolio = sorted(temp_dfs, key = lambda x : x[1].iloc[i]['portfolio_profit'], reverse= True)[0][0] 
        history.loc[i, 'profit'] = profit
    if log:
        print(f'Trading finished. Total profit: {(budget - 10000000) / 10000000 * 100}')

    metrics = pd.DataFrame([], columns = ['percentage'])
    metrics.loc['ap'] = history.loc[history['profit'] > 0, 'profit'].mean()
    metrics.loc['al'] = history.loc[history['profit'] < 0, 'profit'].mean()
    metrics.loc['ar'] = history['profit'].mean()
    metrics.loc['wr'] = len(history[history['profit'] > 0]) / len(history)
    history = history.reset_index(drop= True)
    history['date'] = temp_df['date']

    return history, metrics

In [243]:
history, metrics = backtest_adaptive(monthly_test_data, portfolios)

In [244]:
history['date'] = monthly_test_data[1:]['date'].values

In [245]:
monthly_test_data['profit'] = monthly_test_data.drop('date', axis = 1).mean(axis = 1)

In [246]:
print('Downtrend: 2022 - 2023')
print(f'VN30 average: {total_profit(monthly_test_data[1:13])}')
print(f'Strategy profit: {total_profit(history[0:12])}\n')

print('Uptrend 1: 2023 - 2024:')
print(f'VN30 average: {total_profit(monthly_test_data[13:25])}')
print(f'Strategy profit: {total_profit(history[12:24])}\n')

print('Uptrend 2: 2024 - 2025:')
print(f'VN30 average: {total_profit(monthly_test_data[25:])}')
print(f'Strategy profit: {total_profit(history[24:])}\n')

Downtrend: 2022 - 2023
VN30 average: -26.072728055416906
Strategy profit: -36.0856346896437

Uptrend 1: 2023 - 2024:
VN30 average: 14.455056701177572
Strategy profit: 25.815476786551738

Uptrend 2: 2024 - 2025:
VN30 average: 17.352917371112152
Strategy profit: 26.676677082849576

